In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/train_drug.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/test_features.csv


## importing important libraries, data, and some preprocessing steps

In [2]:
# import libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers, regularizers, Sequential,backend, callbacks, optimizers ,metrics, Model, losses
from tensorflow.keras.layers import Dense, Activation, Input, BatchNormalization
from sklearn.decomposition import PCA
from sklearn.metrics import log_loss
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier

2024-04-25 10:29:22.482302: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 10:29:22.482398: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 10:29:22.618314: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# importing data
train_features = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
train_target_scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')

In [4]:
train_features.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [5]:
# separating cell features and gene features
gene_features = []
cell_features = []
for i in train_features.columns:
    if i.startswith('g-'):
        gene_features.append(i)
    if i.startswith('c-'):
        cell_features.append(i)

print('the length of gene features', len(gene_features))
print('the lenght of cell features', len(cell_features))

train_merge = pd.merge(train_features, train_target_scored, on = 'sig_id', how = 'left')
print('the length of all inputs and target dataset', train_merge.shape)

the length of gene features 772
the lenght of cell features 100
the length of all inputs and target dataset (23814, 1082)


In [6]:
from sklearn.preprocessing import LabelEncoder
x = train_merge.copy()
le = LabelEncoder()
x['cp_type'] = le.fit_transform(x['cp_type'])
x['cp_time'] = le.fit_transform(x['cp_time'])
x['cp_dose'] = le.fit_transform(x['cp_dose'])

test_features['cp_type'] = le.fit_transform(test_features['cp_type'])
test_features['cp_time'] = le.fit_transform(test_features['cp_time'])
test_features['cp_dose'] = le.fit_transform(test_features['cp_dose'])

In [7]:
target_cols = [col for col in train_target_scored.columns if col not in ['sig_id']]
y = x[target_cols]

In [8]:
x.shape, test_features.shape, y.shape

((23814, 1082), (3982, 876), (23814, 206))

## AutoEncoder

In [9]:
# Autoencoder cell features
# encoder part
tf.random.set_seed(42)
inputs = Input(shape = (100,))
encoder_1 = Dense(90, activation = 'relu')(inputs)
batch_norm = BatchNormalization()(encoder_1)
encoder_2 = Dense(75, activation = 'relu')(batch_norm)

# decoder part
decoder_1 = Dense(75, activation = 'relu')(encoder_2)
batch_norm = BatchNormalization()(decoder_1)
decoder_2 = Dense(90, activation = 'relu')(batch_norm)
batch_norm = BatchNormalization()(decoder_2)
decoder_3 = Dense(100)(encoder_2)

model = tf.keras.Model(inputs = inputs, outputs = decoder_3)
model.compile(optimizer = 'adam', loss = 'mse')
model.fit(x[cell_features], x[cell_features],batch_size = 128, epochs = 200)

Epoch 1/200
102/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.0054

I0000 00:00:1714040982.978876      67 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


187/187 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 1.5477
Epoch 2/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6239
Epoch 3/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5413
Epoch 4/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4854
Epoch 5/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4453
Epoch 6/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4150
Epoch 7/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3891
Epoch 8/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3653
Epoch 9/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3448
Epoch 10/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3279
Epoch 11/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3133
Epoch 12/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2971
Epoch 13/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2829
Epoch 14/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2705
Epoch 15/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step -

In [10]:
encoder = tf.keras.Model(inputs = inputs, outputs = encoder_2)
encoder.save('encoders_cell_features.h5')

In [11]:
# Autoencoder gene features
# encoder part
tf.random.set_seed(42)
inputs = Input(shape = (772,))
encoder_1 = Dense(512, activation = 'relu')(inputs)
batch_norm = BatchNormalization()(encoder_1)
encoder_2 = Dense(420, activation = 'relu')(batch_norm)

# decoder part
decoder_1 = Dense(420, activation = 'relu')(encoder_2)
batch_norm = BatchNormalization()(decoder_1)
decoder_2 = Dense(512, activation = 'relu')(batch_norm)
batch_norm = BatchNormalization()(decoder_2)
decoder_3 = Dense(772)(encoder_2)

model = tf.keras.Model(inputs = inputs, outputs = decoder_3)
model.compile(optimizer = 'adam', loss = 'mse')
model.fit(x[gene_features], x[gene_features], batch_size = 512, epochs = 200)

Epoch 1/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 1.1953
Epoch 2/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6856
Epoch 3/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5964
Epoch 4/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5337
Epoch 5/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4824
Epoch 6/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4359
Epoch 7/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3918
Epoch 8/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3521
Epoch 9/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3193
Epoch 10/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2916
Epoch 11/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2702
Epoch 12/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2545
Epoch 13/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2430
Epoch 14/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2328
Epoch 15/200
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2250
Epo

In [12]:
encoder = tf.keras.Model(inputs = inputs, outputs = encoder_2)
encoder.save('encoders_gene_features.h5')

In [13]:
from tensorflow.keras.models import load_model

encoder = load_model('/kaggle/working/encoders_gene_features.h5')
train_gene_features = encoder.predict(x[gene_features])
test_gene_features = encoder.predict(test_features[gene_features])

encoder = load_model('/kaggle/working/encoders_cell_features.h5')
train_cell_features = encoder.predict(x[cell_features])
test_cell_features = encoder.predict(test_features[cell_features])

745/745 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
745/745 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [14]:
x_1_train = np.hstack((x['cp_type'].values.reshape(-1,1), x['cp_time'].values.reshape(-1, 1),
                       x['cp_dose'].values.reshape(-1, 1),
                       train_gene_features, train_cell_features))

In [15]:
x_1_train.shape

(23814, 498)

In [16]:
x_1_test = np.hstack((test_features['cp_type'].values.reshape(-1,1),
                      test_features['cp_time'].values.reshape(-1, 1),
                      test_features['cp_dose'].values.reshape(-1, 1),
                      test_gene_features, test_cell_features))

In [17]:
x_1_test.shape

(3982, 498)

## Building xgboost model

In [18]:
classifier = MultiOutputClassifier(XGBClassifier(tree_method='gpu_hist', colsample_bytree = 0.6522,
          gamma = 3.6975,
          learning_rate = 0.0503,
          max_delta_step = 2.0706,
          max_depth = 10,
          min_child_weight = 31.5800,
          n_estimators = 166,
          subsample = 0.8639))

In [19]:
y = y.to_numpy()

In [20]:
# here we uses logloss as the given kaggle dataset asked for performing a logloss metrices

def logloss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred, p_min, p_max)
    return -backend.mean(y_true * backend.log(y_pred) + (1 - y_true) * backend.log(1 - y_pred))

In [21]:
# train the model
preds_scores = np.zeros(y.shape)
test_prediction = np.zeros((x_1_test.shape[0], y.shape[1]))
oof_losses = []
NFOLDS = 5
kf = KFold(n_splits = NFOLDS)

for fn, (trn_idx, val_idx) in enumerate(kf.split(x_1_train, y)):

    print('Starting fold: ', fn)

    X_train, X_val = x_1_train[trn_idx], x_1_train[val_idx]
    y_train, y_val = y[trn_idx], y[val_idx]
    
    classifier.fit(X_train, y_train)

    val_preds = classifier.predict_proba(X_val)          
    val_preds = np.array(val_preds)[:, :, 1].T             
    preds_scores[val_idx] = val_preds
    
    loss = log_loss(np.ravel(y_val), np.ravel(val_preds))
    oof_losses.append(loss)

    preds = classifier.predict_proba(x_1_test)
    preds = np.array(preds)[:,:,1].T                            
    test_prediction += preds / NFOLDS
    
print(oof_losses)
print('Mean OOF loss across folds', np.mean(oof_losses))
print('STD OOF loss across folds', np.std(oof_losses))

Starting fold:  0


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:31:30] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:31:31] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:31:32] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserW

Starting fold:  1


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:35:01] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:35:02] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:35:03] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserW

Starting fold:  2


...

Starting fold:  3


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:41:53] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:41:54] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:41:55] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserW

Starting fold:  4


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:45:18] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:45:19] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:45:20] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserW

[0.017345404698251618, 0.01738936678869999, 0.01718296100133302, 0.017259655042894343, 0.017433143641680873]
Mean OOF loss across folds 0.01732210623457197
STD OOF loss across folds 9.017500053117373e-05


In [22]:
print('OOF log loss: ', log_loss(np.ravel(y), np.ravel(preds_scores)))

OOF log loss:  0.017322100942777308


In [23]:
sub = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

sub.iloc[:,1:] = test_prediction
# sub.to_csv('submission.csv', index=False)
sub.head()
sub.to_csv('submission.csv', index=False)

In [24]:
sub.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.001883,0.001884,0.001875,0.013264,0.019817,0.003085,0.002390,0.005757,0.001881,...,0.001883,0.001843,0.002216,0.001912,0.002486,0.001861,0.002675,0.001885,0.002077,0.001900
1,id_001897cda,0.001883,0.001884,0.001875,0.003759,0.005124,0.003193,0.002575,0.004624,0.001881,...,0.001883,0.001822,0.002140,0.001576,0.003897,0.001861,0.043876,0.001885,0.002099,0.001919
2,id_002429b5b,0.001883,0.001884,0.001875,0.003632,0.009597,0.002501,0.002281,0.002646,0.001881,...,0.001883,0.001886,0.002140,0.004465,0.002734,0.001861,0.004440,0.001875,0.001607,0.001910
3,id_00276f245,0.001883,0.001884,0.001875,0.006470,0.005562,0.002476,0.002138,0.003636,0.001881,...,0.001883,0.001876,0.002053,0.010646,0.002958,0.001861,0.001509,0.001875,0.001775,0.001900
4,id_0027f1083,0.001883,0.001884,0.001875,0.006143,0.011330,0.002668,0.002594,0.003150,0.001881,...,0.001883,0.001874,0.002101,0.004105,0.002332,0.001861,0.004627,0.001875,0.001732,0.001891


In [25]:
import pickle
pickle.dump((classifier), open('xgboost+autoencoder.pkl', 'wb'))

![](score.png)